# Training Process
This run has an example of running a training job

1. Creating AI template and instance

In [1]:
import os.path
import shutil

from superai.meta_ai.ai import AITemplate, AI, TrainingOrchestrator
from superai.meta_ai.parameters import Config, HyperParameterSpec, TrainingParameters
from superai.meta_ai.schema import Schema

In [2]:
if os.path.exists(".AISave"):
    shutil.rmtree(".AISave")

In [3]:
template = AITemplate(
    input_schema=Schema(),
    output_schema=Schema(),
    configuration=Config(),
    name="MnistTrainingTemplate",
    description="Template of Sample MNIST training",
    model_class="MnistModel",
    requirements=["tensorflow-gpu==2.3.0", "polyaxon"],
    artifacts={"run": "resources/runDir/run_this.sh"},
    code_path=["resources/runDir"],
)
ai = AI(
    ai_template=template,
    input_params=template.input_schema.parameters(),
    output_params=template.output_schema.parameters(),
    name="mnist_training",
    version=1,
    description="AI instance of sample MNIST training",
)

[02/18/22 17:47:25] INFO     Copying all code_path content - MainThread             ]8;id=1645202845.956445-252679;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:264

                    INFO     Saved model in .AISave/mnist_training/1 - MainThread   ]8;id=1645202845.9690812-691270;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:833

## Create and push training container
Use the following interface to create and push the training container.

> Note: Later, this interface will contain implementation to connect to meta-ai to create a training job.

In [4]:
ai._id = "DAAD6583-1FD7-4718-8B12-333701894FDB".lower()

hyperparams = HyperParameterSpec(epochs=10)
ai.training_deploy(
    orchestrator=TrainingOrchestrator.AWS_EKS,
    skip_build=False,
    enable_cuda=True,
    build_all_layers=True,
    training_parameters=TrainingParameters(hyperparameters=hyperparams)
)

[02/18/22 17:47:27] INFO     Base image                                            ]8;id=1645202847.4730282-361873;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1208
                             'superai-model-s2i-python3711-gpu-seldon:1' found               
                             locally. - MainThread

                    INFO     No change in pip layer. Reusing old layers from image ]8;id=1645202847.482775-523639;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1223
                             sha256:3702826942cecf12d4c049b8ad35753b59b8779e2fdfe6           
                             49687a521c64d44b31... - MainThread

                    INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1645202847.4881501-538054;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1303
                             ai/superai-sdk-private/docs/examples/ai/.AISave/mnist           
                             _training/1/environment -v                                      
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True .                                            
                             superai-model-s2i-python3711-gpu-seldon:1                       
                             mnist_training-pip-layer:1' - MainThread

[02/18/22 17:48:55] INFO     Running 's2i build -E /Users/ankitmishra/Work/mysuper ]8;id=1645202935.159571-544994;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1303
                             ai/superai-sdk-private/docs/examples/ai/.AISave/mnist           
                             _training/1/environment -v                                      
                             /Users/ankitmishra/.aws:/root/.aws -v                           
                             /Users/ankitmishra/.superai:/root/.superai -v                   
                             /Users/ankitmishra/.canotic:/root/.canotic                      
                             --incremental=True . mnist_training-pip-layer:1                 
                             mnist_training:1' - MainThread

[02/18/22 17:48:56] INFO     Built main container `mnist_training:1` - MainThread  ]8;id=1645202936.395926-438812;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/ai.py\ai.py]8;;\:1245

...

                    INFO     Logging in to ECR... - MainThread                ]8;id=1645202937.951545-289711;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:246

[02/18/22 17:49:00] INFO     Tagging to `185169359328.dkr.ecr.us-east-1.amazo ]8;id=1645202940.725766-344046;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:249
                             naws.com/models/dev/daad6583-1fd7-4718-8b12-3337                
                             01894fdb/mnist_training:1` - MainThread

                    INFO     Pushing image... - MainThread                    ]8;id=1645202940.7437272-273734;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:252

Output()

[02/18/22 17:50:39] INFO      Image pushed successfully to 185169359328.dkr.e ]8;id=1645203039.829727-531090;file:///Users/ankitmishra/Work/mysuperai/superai-sdk-private/superai/meta_ai/dockerizer/__init__.py\__init__.py]8;;\:291
                             cr.us-east-1.amazonaws.com/models/dev/daad6583-1                
                             fd7-4718-8b12-333701894fdb/mnist_training:1  -                  
                             MainThread

# Starting training manually
To start training manually, in a separate terminal run, `polyaxon port-forward` to connect to Polyaxon API.

Then run the following cell
> Note: triggering training will be done from meta-ai in the future.

In [14]:
!cat polyaxonfile.yaml

version: 1.1
kind: component
tags: [examples, keras]

inputs:
- {name: conv1_size, type: int, value: 32, isOptional: true}
- {name: conv2_size, type: int, value: 64, isOptional: true}
- {name: dropout, type: float, value: 0.8, isOptional: true}
- {name: hidden1_size, type: int, value: 500, isOptional: true}
- {name: optimizer, type: str, value: adam, isOptional: true}
- {name: log_learning_rate, type: int, value: -3, isOptional: true}
- {name: epochs, type: int, value: 10, isOptional: true}

run:
  kind: job
  container:
    image: 185169359328.dkr.ecr.us-east-1.amazonaws.com/models/dev/daad6583-1fd7-4718-8b12-333701894fdb/mnist_training:1
#    workingDir: "{{ globals.artifacts_path }}/polyaxon-examples/in_cluster/keras/mnist"
    command: ["/opt/conda/envs/env/bin/superai", "ai", "method", "train"]
    imagePullPolicy: Always
    args: ["-p", "/home/model-server/",
           "-tp","/tmp",
           "-mp","/tmp",
           "-m","conv1_size={{ conv1_size }}",
           "-m","conv2_s

In [ ]:
!polyaxon run -f polyaxonfile.yaml -u -l

Creating a new run...
A new run `3d56bfb861634b8e92cd11433e23b37a` was created
You can view this run on Polyaxon UI: http://localhost:8000/ui/default/dev/runs/3d56bfb861634b8e92cd11433e23b37a/
[================================] 958775/958775 - 00:00:00
Artifacts uploaded
Run is approved
Starting logs for run: <Name: None> - <uuid: 3d56bfb861634b8e92cd11433e23b37a>
warning
running
2022-02-18 17:51:29.912739+01:00 | Reading configs from /opt/conda/envs/env/lib/python3.7/site-packages/superai/settings.yaml
2022-02-18 17:51:29.912767+01:00 | Available envs:
2022-02-18 17:51:29.912770+01:00 | - local
2022-02-18 17:51:29.912773+01:00 | - dev
2022-02-18 17:51:29.912775+01:00 | - sandbox
2022-02-18 17:51:29.912777+01:00 | - stg
2022-02-18 17:51:29.912779+01:00 | - prod
2022-02-18 17:51:29.912781+01:00 | Reading configs from /opt/conda/envs/env/lib/python3.7/site-packages/superai/settings.yaml
2022-02-18 17:51:29.912783+01:00 | [02/18/22 16:51:29] INFO     Setting config : prod - MainThread    

2022-02-18 17:51:34.033765+01:00 | Epoch 1/10
2022-02-18 17:51:34.033863+01:00 | 2022-02-18 16:51:34.033782: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
2022-02-18 17:51:34.033904+01:00 | 2022-02-18 16:51:34.033844: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1441] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
  1/600 [..............................] - ETA: 0s - loss: 2.3247 - accuracy: 0.0700[02/18/22 16:51:34] WARNING  From /opt/conda/envs/env/lib/pyt deprecation.py:323
2022-02-18 17:51:34.167184+01:00 |                              hon3.7/site-packages/tensorflow/
2022-02-18 17:51:34.167200+01:00 |                              python/ops/summary_ops_v2.py:127
2022-02-18 17:51:34.167204+01:00 |                              7: stop (from tensorflow.python.
2022-02-18 17:51:34.167208+01:00 |                              